### 库

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from tqdm.notebook import tqdm, trange, tnrange
import json, re

In [2]:
import numpy as np
import pandas as pd
import os, sys
from time import time, sleep

In [3]:
from vVariableInspector import _getshapeof
from vUtil.vFile import fprint, readlines, linesReader
from vUtil.vEmail import sendEmail
from vUtil.vTime import convertSeconds, getNow

In [4]:
from vMysql import MysqlProxy

In [5]:
from util import frmt, rmUnseen, groupConcat, deleteIndexRows
from cfg import host, master, port, contType
from cfg import dbHost, dbUser, dbPwd
from cfg import ncols

### mysql、es

In [6]:
index = 'landinn_project'

In [7]:
es=Elasticsearch(hosts=host,port=port)

In [8]:
db = MysqlProxy(ip=dbHost, user=dbUser, password=dbPwd)

### 函数

In [9]:
sTmpProject = '''
create temporary table tmp_project
(
    select golaxy_project_id as gid, 
        ifnull(project_title, project_title_en) as title,
        ifnull(discipline_first, discipline_first_en) as d1,
        ifnull(discipline_secondary, discipline_secondary_en) as d2,
        ifnull(discipline_tertiary, discipline_tertiary_en) as d3,
        cast(start_year as char) as start_year,
        cast(end_year as char) as end_year
    from project
    where !ifnull(is_deleted, 0)
    limit %d, %d
);
'''
sRmTmpProject = '''
drop temporary table tmp_project;
'''

In [10]:
sSelectPublish = '''
select gid, cast(author_id as char) as scholar_id
from
    tmp_project as a
left join project_authors as b
on a.gid = b.project_id
where !ifnull(b.is_deleted, 0);
'''

In [11]:
sSelectData = '''
select  gid, title, 
        if (d1 is null and d2 is null and d3 is null, null, 
            concat('{', ifnull(d1, ''), '}{', ifnull(d2, ''), '}{', ifnull(d3, ''),'}')
        ) as discipline,
        ifnull(project_description, project_description_en) as description,
        start_year, end_year
from
    tmp_project as a
left join project_abstracts as b 
on a.gid = b.project_id
where !ifnull(b.is_deleted, 0);
'''

In [12]:
def getActions (data, publish):
    actions = []
    
    p2s = {publish['gid'][i] : publish['scholar_id'][i] for i in range(len(publish))}
    
    for i in trange(len(data), leave=False):
        if data['gid'][i] not in p2s: continue
        action={'_op_type':'index',#操作 index update create delete  
            '_index':index,#index
            '_id' : data['gid'][i],
            '_source':
           {
                "id" : data['gid'][i],
                "title" : rmUnseen(data['title'][i]),
                "discipline" : rmUnseen(data['discipline'][i]),
                "description" : rmUnseen(data['description'][i]),
                "scholars" : p2s[data['gid'][i]],
                "start_year" : data['start_year'][i],
                "end_year" : data['end_year'][i]
            }
        }
        actions.append(action)
    return actions

### 索引

In [13]:
table = 'project'

In [14]:
deleteIndexRows(db, es, table, 'golaxy_project_id', index=index)

{'landinn_project': 0}

In [17]:
sizeBulk = 100
now = 0
nProject = db.count(table, where='!ifnull(is_deleted,0)').values.item()

In [14]:
# now = 120

In [18]:
now, sizeBulk, nProject

(0, 100, 213313)

In [19]:
startOfAll = time()
tr = trange(now, nProject, sizeBulk, ncols=ncols)
for i in tr:
    tr.set_description(f'({getNow()}){i}')
    now = i
    
    db.sql(sTmpProject % (now, sizeBulk))
    
    data = db.sql(sSelectData)
    publish = db.sql(sSelectPublish)
    db.sql(sRmTmpProject)
    db.close()
    
    actions = getActions(data, groupConcat(publish, 'gid', ';'))
    if actions: helpers.bulk(client=es,actions=actions)
sendEmail(f'insert cost time {convertSeconds(time() - startOfAll)}', 'insert landinn es projects complete')

### 关闭

In [10]:
es.close()

In [39]:
db.close()